In [0]:
pip install azure-storage-blob

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
##################################################################
# Codigo que funciona correctamente
##################################################################

from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd
import io
import pyspark
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
#spark = pyspark.sql.SparkSession.builder.appName("Connect to MySQL").getOrCreate()
spark = SparkSession.builder.appName("EjemploDatabricks").getOrCreate()

# Información de tu cuenta de almacenamiento y el folder
storage_account_name = "datalakeproyectoreviews"
storage_account_access_key = "nUJSljEWbc2ePVYZkrbcBkXChgXK4QMHTS5vVuCnyjZIxQSbxTKzJmfQqMq2c5G92zGOrNeFzA+w+AStjnDikg=="
container_name = "datalakereviews"
folder_path = "metadata_sitios/"

# Conecta al servicio de Blob Storage
connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_access_key};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Obtén una referencia al contenedor
container_client = blob_service_client.get_container_client(container_name)

# Lista los blobs en el folder
blobs = container_client.list_blobs(name_starts_with=folder_path)

# Obtiene el nombre del último archivo (blob)
last_blob_name = None
last_modified = None

for blob in blobs:
    if last_modified is None or blob['last_modified'] > last_modified:
        last_blob_name = blob['name']
        last_modified = blob['last_modified']

# Elimina la extensión duplicada ".json" si existe
if last_blob_name is not None:
    last_blob_name = last_blob_name.replace(".json.json", ".json")

if last_blob_name is not None:
    print("Nombre del último archivo:", last_blob_name)
    
    # Descarga el contenido del archivo desde Azure Blob Storage
    blob_client = container_client.get_blob_client(blob=last_blob_name)
    json_data = blob_client.download_blob()
    json_text = json_data.readall()

    # Carga el contenido JSON en un DataFrame de Pandas
    df = pd.read_json(io.BytesIO(json_text), lines=True)

    
    # Ahora puedes trabajar con df como un DataFrame de Pandas
    # print(df.head())  # Esto imprimirá las primeras filas del DataFrame
else:
    raise Exception("No se encontraron archivos en la carpeta.")

Nombre del último archivo: metadata_sitios/8.json


In [0]:


# en cada options levanto una tabla
con_business = {
  "host": "reviewsdatabase.mysql.database.azure.com",
  "port": 3306,
  "database": "reviews",
  "dbtable": "business",
  "user": "reviewsadmin",
  "password": "Our_password@2023",
  "require_secure_transport": 'On',
  "useSSL": True
}

con_catego = {
  "host": "reviewsdatabase.mysql.database.azure.com",
  "port": 3306,
  "database": "reviews",
  "dbtable": "categories",
  "user": "reviewsadmin",
  "password": "Our_password@2023",
  "require_secure_transport": 'On',
  "useSSL": True
}

con_busi_catego = {
  "host": "reviewsdatabase.mysql.database.azure.com",
  "port": 3306,
  "database": "reviews",
  "dbtable": "bussiness_category",
  "user": "reviewsadmin",
  "password": "Our_password@2023",
  "require_secure_transport": 'On',
  "useSSL": True
}

options4 = {
  "host": "reviewsdatabase.mysql.database.azure.com",
  "port": 3306,
  "database": "reviews",
  "dbtable": "misc_bussin",
  "user": "reviewsadmin",
  "password": "Our_password@2023",
  "require_secure_transport": 'On',
  "useSSL": True
}

#df_business = spark.read.format("mysql").options(**options1).load()
#df_categories = spark.read.format("mysql").options(**options2).load()
#df_bussiness_category = spark.read.format("mysql").options(**options3).load()
#df_misc_bussin = spark.read.format("mysql").options(**options4).load()

In [0]:
def business_exist(gmap):
    qsele = {
        "host": "reviewsdatabase.mysql.database.azure.com",
        "port": 3306,
        "database": "reviews",
        "query": f"select gmap_id from business where gmap_id = '{gmap}'",
        "user": "reviewsadmin",
        "password": "Our_password@2023",
        "require_secure_transport": 'On',
        "useSSL": True
        }
    df_sele = spark.read.format("mysql").options(**qsele).load()
    return df_sele.count() # si retorna 0 no encontró gmap_id en la tabla


In [0]:
def catego_exist(cate):
    qsele = {
        "host": "reviewsdatabase.mysql.database.azure.com",
        "port": 3306,
        "database": "reviews",
        "query": f"select id_category from categories where name_category = '{cate}'",
        "user": "reviewsadmin",
        "password": "Our_password@2023",
        "require_secure_transport": 'On',
        "useSSL": True
        }
    df_sele = spark.read.format("mysql").options(**qsele).load()
    return df_sele.count() # si retorna 0 no encontró la categoria en la tabla

In [0]:
def estado_retorna(lis):
    # Dividir la cadena en partes separadas por comas
    if lis == None:
        return None
        
    partes = lis.split(',')

    # Si hay al menos cuatro partes (tres comas), retornar las primeras 2 posiciones de la cuarta parte
    if len(partes) >= 4:
        cuarta_parte = partes[3]
        estado_corto = cuarta_parte[:3]
        return estado_corto.strip()

In [0]:
def inserta_categoria(categoria):
    qsele = {
    "host": "reviewsdatabase.mysql.database.azure.com",
    "port": 3306,
    "database": "reviews",
    "query": "select insert_category('categoria')",
    "user": "reviewsadmin",
    "password": "Our_password@2023",
    "require_secure_transport": 'On',
    "useSSL": True
        }
    df_sele = spark.read.format("mysql").options(**qsele).load()
    valor_row = valor.select("insert_category('categoria')").collect()[0]
    valor_categoria = valor_row["insert_category('categoria')"]
    return valor_categoria    


In [0]:
def inserta_busi_catego(gmap_id,id_category):
    df_sele = spark.createDataFrame([
                (gmap_id, id_category)
                ])
   
    df_sele.write.format("mysql").options(**con_busi_catego).mode("append").saveAsTable("business_category")


In [0]:
def inserta_business(gmap_id, name, address, latitude, longitude, url, state_corto):
    ur = "jdbc:mysql://reviewsdatabase.mysql.database.azure.com:3306/reviews?requireSSL=true"
    qsele = {
        "url": ur,
        "dbtable": "business",
        "user": "reviewsadmin",
        "password": "Our_password@2023",
        "driver": "com.mysql.jdbc.Driver"
    }
    
    schema = StructType([
        StructField("gmap_id", StringType(), nullable=False),
        StructField("name", StringType(), nullable=False),
        StructField("address", StringType(), nullable=True),
        StructField("latitude", StringType(), nullable=False),
        StructField("longitude", StringType(), nullable=False),
        StructField("url", StringType(), nullable=False),
        StructField("state_corto", StringType(), nullable=False)
    ])
    
    data = [(gmap_id, name, address, float(latitude), float(longitude), url, state_corto)]
    
    # Crear el DataFrame utilizando el esquema y la sesión 'spark' de Databricks
    df_insert = spark.createDataFrame(data, schema=schema)
    
    # Escribir en la base de datos usando el formato "jdbc"
    df_insert.write.format("jdbc").options(**qsele).mode("append").save()

In [0]:
df_bussi_error = pd.DataFrame(columns=['gmap_id,name,address,latitude,longitude,relative_results,url'])

In [0]:
for index, row in df.iterrows():
    cat_str = row["category"]
    gmap_id = row["gmap_id"]
    name = row["name"]
    address = row["address"]
    latitude = row["latitude"]
    longitude = row["longitude"]
    category_value = row["category"]
    relative_results = row["relative_results"]
    url = row["url"]
    state_corto = estado_retorna(address)
    if address is None:
        address = "No se conoce"
        state_corto = 'XX'
    
    if name == address:
        # Hay una falla en el dataset donde algunos name y address son iguales
        # Obtengo el nombre del business
        
        if ',' in name:
            posicion_coma = name.index(',')
            name = name[0:posicion_coma]
        else:
            # Si no hay una coma en 'name', asegúrate de manejar este caso de manera adecuada
            name = 'No se conoce el nombre'
            state_corto = 'XX'

        # Obtengo la dirección    
        concurrencia = address.count(',')
        if concurrencia > 0:
            partes = address.split(',')
            address = ','.join(partes[-3:])
        else:
            address = 'No se conoce la dirección'
    
    if len(name) > 250:
            name = name[:250]
    
    if business_exist(gmap_id) == 0:
        # compruebo si no encontro state_corto o no address estaba None
        if state_corto is None:
            state_corto = 'XX'

        # Tabla Business: insertamos el registro
        inserta_business(gmap_id, name, address, latitude, longitude, url, state_corto)
        # Tabla Categoria
        df_busi_cat = pd.DataFrame(columns=['gmap_id, categoria, id_category'])
        if isinstance(category_value,(np.ndarray)):
            for value in category_value:
                valor = str(value[0])
                new_row_cat = {
                                "gmap_id": gmap_id,
                                "categoria": valor
                                }
                rows_busi_cat.append(new_row_cat)
            else:
                new_row_cat = {
                                "gmap_id": gmap_id,
                                "categoria": valor
                               }
                rows_busi_cat.append(new_row_cat)
        
        for index, row in df_busi_cat.iterrows():
            categoria = row['categoria']
            gmap_id = row['gmap_id'] 
            if catego_exist(categoria) > 0:
                continue
            else:
                valor = inserta_categoria(categoria)
                row['id_category'] = valor
                inserta_busi_catego(gmap_id,valor)

spark.stop()

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-1447499638285740>:15
     11 state_corto = '--'#estado_retorna(address)
     12 if name == address:
     13     #hay una falla en el dataset donde algunos name y address son iguales
     14     #obtengo el nombre del business
---> 15     posicion_coma = name.index(',')
     16     name = name[0:posicion_coma]
     18     #obtengo la dirección    

ValueError: substring not found